In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName(name='Project').getOrCreate()

In [3]:
df = spark.read.csv(path='./Data/walmart_stock.csv', inferSchema=True, header=True)

In [5]:
df.show(10)

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [6]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [8]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [11]:
df.select(['Date', 'Open', 'Close', 'Volume']).show(10)

+----------+------------------+------------------+--------+
|      Date|              Open|             Close|  Volume|
+----------+------------------+------------------+--------+
|2012-01-03|         59.970001|         60.330002|12668800|
|2012-01-04|60.209998999999996|59.709998999999996| 9593300|
|2012-01-05|         59.349998|         59.419998|12768200|
|2012-01-06|         59.419998|              59.0| 8069400|
|2012-01-09|         59.029999|             59.18| 6679300|
|2012-01-10|             59.43|59.040001000000004| 6907300|
|2012-01-11|         59.060001|         59.400002| 6365600|
|2012-01-12|59.790001000000004|              59.5| 7236400|
|2012-01-13|             59.18|59.540001000000004| 7729300|
|2012-01-17|         59.869999|         59.849998| 8500000|
+----------+------------------+------------------+--------+
only showing top 10 rows



In [12]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [13]:
from pyspark.sql.functions import format_number

In [14]:
summary = df.describe()

In [15]:
summary.select(summary['summary'],
              format_number(col=summary['Open'].cast('float'),d=2).alias('Open'),
              format_number(summary['High'].cast('float'),2).alias('High'),
              format_number(summary['Low'].cast('float'),2).alias('Low'),
              format_number(summary['Close'].cast('float'),2).alias('Close'),
              summary['Volume'].cast('int').alias('Volume')).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



In [20]:
df.withColumn(colName='HV Ratio', col=df['High']/df['Volume']).select(['Date', 'High', 'Low', 'HV Ratio']).show(10)

+----------+------------------+------------------+--------------------+
|      Date|              High|               Low|            HV Ratio|
+----------+------------------+------------------+--------------------+
|2012-01-03|         61.060001|         59.869999|4.819714653321546E-6|
|2012-01-04|         60.349998|         59.470001|6.290848613094555E-6|
|2012-01-05|         59.619999|         58.369999|4.669412994783916E-6|
|2012-01-06|         59.450001|         58.869999|7.367338463826307E-6|
|2012-01-09|         59.549999|         58.919998|8.915604778943901E-6|
|2012-01-10|59.709998999999996|             58.98|8.644477436914568E-6|
|2012-01-11|         59.529999|59.040001000000004|9.351828421515645E-6|
|2012-01-12|              60.0|         59.400002| 8.29141562102703E-6|
|2012-01-13|59.610001000000004|59.009997999999996|7.712212102001476E-6|
|2012-01-17|60.110001000000004|             59.52|7.071764823529412E-6|
+----------+------------------+------------------+--------------

In [22]:
df.orderBy(df['High'].desc()).show(10)

+----------+-----------------+-----------------+-----------------+---------+--------+-----------------+
|      Date|             Open|             High|              Low|    Close|  Volume|        Adj Close|
+----------+-----------------+-----------------+-----------------+---------+--------+-----------------+
|2015-01-13|        90.800003|        90.970001|            88.93|89.309998| 8215400|        83.825448|
|2015-01-08|        89.209999|90.66999799999999|            89.07|90.470001|12713600|84.91421600000001|
|2015-01-09|            90.32|        90.389999|            89.25|89.349998| 8522500|        83.862993|
|2015-01-12|        89.360001|        90.309998|        89.220001|90.019997| 7372500|        84.491846|
|2015-01-23|88.41999799999999|        89.260002|        87.889999|88.510002| 7565800|83.07458100000001|
|2015-01-26|        88.309998|        89.160004|        88.120003|88.629997| 4666700|        83.187207|
|2015-01-07|        86.779999|            88.68|86.6699979999999

In [31]:
from pyspark.sql.functions import mean, max, min, count

In [25]:
df.select(mean('Close').alias('Average Closing')).show()

+-----------------+
|  Average Closing|
+-----------------+
|72.38844998012726|
+-----------------+



In [28]:
df.select(max('Volume').alias('Max Volume'), min('Volume').alias('Min Volume')).show()

+----------+----------+
|Max Volume|Min Volume|
+----------+----------+
|  80898100|   2094900|
+----------+----------+



In [29]:
df.filter('Close < 60').count()

81

In [30]:
df.filter(df['Close'] < 60).count()

81

In [38]:
(df.filter(df['High'] > 80).count() / df.count()) * 100

9.141494435612083

In [39]:
from pyspark.sql.functions import corr

In [40]:
df.select(corr(col1='High', col2='Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [43]:
from pyspark.sql.functions import year

df.withColumn('Year', year(df['Date'])).groupby('Year').max().show()

+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
|Year|        max(Open)|max(High)| max(Low)|max(Close)|max(Volume)|   max(Adj Close)|max(Year)|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
|2015|        90.800003|90.970001|    89.25| 90.470001|   80898100|84.91421600000001|     2015|
|2013|        81.209999|81.370003|    80.82| 81.209999|   25683700|        73.929868|     2013|
|2014|87.08000200000001|88.089996|86.480003| 87.540001|   22812400|81.70768000000001|     2014|
|2012|        77.599998|77.599998|76.690002| 77.150002|   38007300|        68.568371|     2012|
|2016|             74.5|75.190002|73.629997| 74.300003|   35076700|        73.233524|     2016|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+



In [45]:
add_yearColumn = df.withColumn(colName='Year', col=year(df['Date']))

In [47]:
add_yearColumn.show(10)

+----------+------------------+------------------+------------------+------------------+--------+------------------+----+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+--------+------------------+----+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|2012|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|2012|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|2012|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|2012|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|2012|
|2012-01-10|            

In [56]:
max_year = add_yearColumn.groupBy('Year').max()

In [57]:
max_year.show()

+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
|Year|        max(Open)|max(High)| max(Low)|max(Close)|max(Volume)|   max(Adj Close)|max(Year)|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
|2015|        90.800003|90.970001|    89.25| 90.470001|   80898100|84.91421600000001|     2015|
|2013|        81.209999|81.370003|    80.82| 81.209999|   25683700|        73.929868|     2013|
|2014|87.08000200000001|88.089996|86.480003| 87.540001|   22812400|81.70768000000001|     2014|
|2012|        77.599998|77.599998|76.690002| 77.150002|   38007300|        68.568371|     2012|
|2016|             74.5|75.190002|73.629997| 74.300003|   35076700|        73.233524|     2016|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+



In [53]:
max_year.select(['Year', 'max(High)']).show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



In [54]:
from pyspark.sql.functions import month

In [55]:
month_df = df.withColumn(colName='Month', col=month('Date'))

In [58]:
month_avgs = month_df.select(['Month', 'Close']).groupBy('Month').mean()

In [59]:
month_avgs.show()

+-----+----------+-----------------+
|Month|avg(Month)|       avg(Close)|
+-----+----------+-----------------+
|   12|      12.0|72.84792478301885|
|    1|       1.0|71.44801958415842|
|    6|       6.0| 72.4953774245283|
|    3|       3.0|71.77794377570092|
|    5|       5.0|72.30971688679247|
|    9|       9.0|72.18411785294116|
|    4|       4.0|72.97361900952382|
|    8|       8.0|73.02981855454546|
|    7|       7.0|74.43971943925233|
|   10|      10.0|71.57854545454543|
|   11|      11.0| 72.1110893069307|
|    2|       2.0|  71.306804443299|
+-----+----------+-----------------+



In [60]:
month_avgs.select(['Month','avg(Close)']).orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

